In [1]:
import os
import numpy as np

import pandas as pd
import tensorflow as tf

from transformers import BertTokenizer, RobertaTokenizer
from BertModel import BertForQALabeling
from RoBERTaModel import *
from Trainer import Trainer
from preprocessing import dataPreprocessor
from parameters import *

import horovod.tensorflow as hvd

In [2]:
        hvd.init()
        gpus = tf.config.list_physical_devices('GPU') 
        for gpu in gpus:
            print(gpu)
            tf.config.experimental.set_memory_growth(gpu, True)
        if gpus:
            print("gpus ", gpus)
            print("local rank ",hvd.local_rank())
            tf.config.set_visible_devices(gpus[hvd.local_rank()], 'GPU')
            print(tf.config.get_visible_devices())

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')
gpus  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]
local rank  0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
data_dir = "google-quest-challenge/"

train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))
test_df = pd.read_csv(os.path.join(data_dir, "test.csv"))
submit_df = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
stack_df = pd.read_csv(os.path.join(data_dir, "stackexchange_data.csv"))
stack_df.head()

,id,host,question_username,question_score,question_views,question_favs,answers_count,answers_max_score,answers_mean_score,question_title,question_body,answer_username,answer,answer_score,is_answer_accepted
0,1,3dprinting.stackexchange.com,Adam Davis,3.767810,3.912607,0.000000,3.188060,3.749319,3.431946,How to obtain high resolution prints in a shor...,When I've printed an object I've had to choose...,hroncok,You could experiment with slicing. For example...,3.213366,2.07692
1,1,3dprinting.stackexchange.com,Adam Davis,3.767810,3.912607,0.000000,3.188060,3.749319,3.431946,How to obtain high resolution prints in a shor...,When I've printed an object I've had to choose...,plaintoothpaste,For FDM technologies in general with a single ...,1.606683,0.00000
2,4,3dprinting.stackexchange.com,Adam Davis,4.541637,3.921554,2.779972,4.670423,3.320479,3.431946,Are there any metals that exhibit a large glas...,Plastic is used in 3D FDM/FFF printing partly ...,TextGeek,"I""m no expert on this, but the article at http...",2.620387,0.00000
3,4,3dprinting.stackexchange.com,Adam Davis,4.541637,3.921554,2.779972,4.670423,3.320479,3.431946,Are there any metals that exhibit a large glas...,Plastic is used in 3D FDM/FFF printing partly ...,Ryan Carlyle,A few things are required for effective extrus...,2.845828,2.07692
4,2,3dprinting.stackexchange.com,kenorb,5.291027,5.526324,3.507934,5.199501,5.195138,4.521993,Is 3D printing safe for your health?,"I would like to buy a 3D printer, but I'm conc...",Tom van der Zanden,There is very little information about safety ...,4.452511,2.07692


In [4]:
train_X_df = train_df[['question_title', 'question_body', 'answer']]
train_targets_df = train_df[["question_asker_intent_understanding",
    "question_body_critical",
    "question_conversational",
    "question_expect_short_answer",
    "question_fact_seeking",
    "question_has_commonly_accepted_answer",
    "question_interestingness_others",
    "question_interestingness_self",
    "question_multi_intent",
    "question_not_really_a_question",
    "question_opinion_seeking",
    "question_type_choice",
    "question_type_compare",
    "question_type_consequence",
    "question_type_definition",
    "question_type_entity",
    "question_type_instructions",
    "question_type_procedure",
    "question_type_reason_explanation",
    "question_type_spelling",
    "question_well_written",
    "answer_helpful",
    "answer_level_of_information",
    "answer_plausible",
    "answer_relevance",
    "answer_satisfaction",
    "answer_type_instructions",
    "answer_type_procedure",
    "answer_type_reason_explanation",
    "answer_well_written"]]

In [5]:
q_title = train_X_df['question_title'].values
q_body = train_X_df['question_body'].values
answer = train_X_df['answer'].values

targets = train_targets_df.to_numpy()

stack_q_title = stack_df['question_title'].values
stack_q_body = stack_df['question_body'].values
stack_answer = stack_df['answer'].values

In [6]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

dataPreprocessor.logger = False
dataPreprocessor.tokenizer = tokenizer
dataPreprocessor.model = "Roberta"

In [7]:
preprocessedInput = dataPreprocessor.preprocessBatch(q_body, q_title, answer, max_seq_lengths=(26,260,210,500))

Token indices sequence length is longer than the specified maximum sequence length for this model (881 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (898 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2900 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (848 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (976 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

In [8]:
preprocessedStack = dataPreprocessor.preprocessBatch(stack_q_body, stack_q_title, stack_answer, max_seq_lengths=(26,260,210,500))

Token indices sequence length is longer than the specified maximum sequence length for this model (800 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (646 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (770 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (958 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (613 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [9]:
print(preprocessedInput.shape)
print(targets.shape)

print(preprocessedStack.shape)

(6079, 3, 500)
(6079, 30)
(3899, 3, 500)


In [10]:
Trainer.train(model=RoBERTaForQALabelingMultipleHeads.from_pretrained('roberta-base', num_labels=num_labels, output_hidden_states=True), 
              tokenizer=tokenizer, 
              preprocessedInput=preprocessedInput, 
              targets=targets, 
              preprocessedPseudo=preprocessedStack)

starting training for 2 epochs


TypeError: unsupported operand type(s) for /: 'generator' and 'int'